# Lab 1: Multi-Layer Perceptron - Clasificación de Calidad del Vino

**Objetivo:** Construir un MLP personalizado con PyTorch para clasificar vinos en 3 categorías:
- Baja (0): 3-5
- Media (1): 6-7  
- Alta (2): 8-9

## 1. Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Semillas para reproducibilidad
np.random.seed(42)
torch.manual_seed(42)

## 2. Carga y EDA

In [ ]:
file_path = '/home/jalasop/Documentos/Semester_5/DeepLearning/repos/csai-353-deep-learning-vault/csds-353-deep-learning-vault/notebooks/data/wine-quality-dataset.csv'

try:
    df = pd.read_csv(file_path)
except:
    df = pd.read_csv(file_path, sep=';')

df.head()

In [ ]:
print(f'Shape: {df.shape}')
print(f'Columnas: {list(df.columns)}')
print(f'Valores faltantes: {df.isnull().sum().sum()}')

### Distribución de calidad y conversión a clases

In [ ]:
# Función para convertir puntuaciones en 3 clases
def classify_quality(quality):
    if quality <= 5:
        return 0  # Baja
    elif quality <= 7:
        return 1  # Media
    else:
        return 2  # Alta

df['quality_class'] = df['quality'].apply(classify_quality)

# Visualización de distribuciones
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

df['quality'].value_counts().sort_index().plot(kind='bar', ax=axes[0])
axes[0].set_title('Distribución Original')
axes[0].set_xlabel('Quality Score')

df['quality_class'].value_counts().sort_index().plot(kind='bar', ax=axes[1], color=['red', 'orange', 'green'])
axes[1].set_title('Distribución por Clases')
axes[1].set_xticklabels(['Baja', 'Media', 'Alta'], rotation=0)

plt.tight_layout()
plt.show()

print(df['quality_class'].value_counts())

**Análisis:** Se observa un desbalance significativo en las clases en este caso la clase Media  domina con 665 muestras 55%, seguida de Baja (0) con 522 muestras 43%, mientras que la clase Alta (2) solo tiene 16 muestras 2 por ciento. pieso aqui hay un desbalance lo que puede resultartle dificil al modelo apra evaluar y preveer los valore de vino premium

### Correlaciones con calidad

In [ ]:
# Calcular correlaciones con la variable objetivo
features = df.columns[:-2]
correlations = df[features].corrwith(df['quality_class']).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
correlations.plot(kind='barh')
plt.title('Correlación de características con calidad')
plt.xlabel('Correlación')
plt.tight_layout()
plt.show()

print(correlations)

**Análisis:** El alcohol muestra la correlación positiva más fuerte (+0.455), ilo que nos dice  que los vinos con mayor graduación tienden a ser de mejor calidad.

La acidez volátil tiene correlación negativa (-0.332), ya que investigfando un poco los niveles altos sugieren que puede haber deterioroy por ultimo los sulfatos (+0.240) también correlacionan positivamente. Estas características serán clave para el modelo.

## 3. Preprocesamiento de Datos

**¿Cómo transformamos el dataset?**

Aplicamos las siguientes transformaciones:

1. **Conversión de etiquetas**: Transformamos las puntuaciones de calidad (3-9) en 3 clases balanceadas
2. **División estratificada**: Separamos los datos manteniendo la proporción de clases en cada conjunto (70% train, 15% val, 15% test)
3. **Normalización**: Aplicamos StandardScaler para que todas las características tengan media=0 y std=1
4. **Conversión a tensores**: Preparamos los datos para PyTorch (FloatTensor para X, LongTensor para y)

In [ ]:
# Separar características (X) y etiquetas (y)
X = df[features].values
y = df['quality_class'].values

# División estratificada: mantiene proporciones de clases
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42, stratify=y_temp)

print(f'Train: {len(X_train)} muestras')
print(f'Val: {len(X_val)} muestras')
print(f'Test: {len(X_test)} muestras')

**Normalización con StandardScaler**

Sin normalizacion  ejemplo  características como 'total sulfur dioxide' (rango: 6-289) dominarían es decir se tomarian mas encuenta  sobre 'pH' (rango: 2.7-4.0). StandardScaler garantiza que todas las características contribuyan al aprendizaje.

In [ ]:
# Normalizar: media=0, std=1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit solo en train
X_val_scaled = scaler.transform(X_val)  # Transform en val y test
X_test_scaled = scaler.transform(X_test)



In [ ]:
# Convertir a tensores PyTorch
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test)

# Crear DataLoaders para batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32)



## 4. Implementación del MLP

**Arquitectura del modelo:**
- Input: 11 nodos (11 características del vino)
- Hidden 1: 64 neuronas + ReLU
- Hidden 2: 32 neuronas + ReLU
- Output: 3 nodos (3 clases) + log_softmax

In [ ]:
class CustomMLP(nn.Module):
    def __init__(self, input_size, hidden1=64, hidden2=32, num_classes=3):
        super(CustomMLP, self).__init__()
        # Definir capas
        self.fc1 = nn.Linear(input_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, num_classes)

        # Inicialización Xavier para mejor convergencia
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        # Forward pass: capa -> activación
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.log_softmax(x, dim=1)  # Log-probabilidades para NLLLoss
        return x

# Instanciar modelo
input_size = X_train_tensor.shape[1]
model = CustomMLP(input_size=input_size)

print(model)
print(f'\nParámetros totales: {sum(p.numel() for p in model.parameters()):,}')

## 5. Entrenamiento

In [ ]:
# Configurar función de pérdida y optimizador
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)


In [ ]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calcular métricas
        total_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return total_loss / total, correct / total

def validate_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return total_loss / total, correct / total

In [ ]:
# Bucle de entrenamiento con early stopping
num_epochs = 100
patience = 15
best_val_loss = float('inf')
patience_counter = 0

history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print(f"{'Época':<8} {'Train Loss':<12} {'Train Acc':<12} {'Val Loss':<12} {'Val Acc':<12}")
print('='*60)

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate_epoch(model, val_loader, criterion)

    # Guardar historial
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    scheduler.step(val_loss)

    if (epoch + 1) % 10 == 0:
        print(f"{epoch+1:<8} {train_loss:<12.4f} {train_acc:<12.4f} {val_loss:<12.4f} {val_acc:<12.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'\nEarly stopping en época {epoch+1}')
            break

# Cargar el mejor modelo
model.load_state_dict(best_model_state)
print(f'\nMejor val loss: {best_val_loss:.4f}')

### Curvas de aprendizaje

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Validation')
axes[0].set_title('Loss')
axes[0].set_xlabel('Época')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['val_acc'], label='Validation')
axes[1].set_title('Accuracy')
axes[1].set_xlabel('Época')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Análisis:** Las curvas muestran convergencia rápida en las primeras 10 epocas, alcanzando 98% de accuracy. Las curvas de train y validation permanecen cercanas lo que inndica la  ausencia de overfitting.

El early stopping activó en la época 54 mas o menos previniendo entrenamiento innecesario.

## 6. Evaluación

In [ ]:
# Predicciones en conjunto de prueba
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predictions = torch.max(test_outputs, 1)

y_pred = test_predictions.numpy()
y_true = y_test_tensor.numpy()

# Calcular métricas
test_accuracy = accuracy_score(y_true, y_pred)
print(f'\nAccuracy en Test: {test_accuracy:.4f}\n')

class_names = ['Baja (3-5)', 'Media (6-7)', 'Alta (8-9)']
print(classification_report(y_true, y_pred, target_names=class_names))

**Análisis:** El modelo alcanzó 99.42% accuracy en test. Las clases Baja y Media tienen precision y recall muy acertados (1.00). La clase Alta muestra recall de 0.50 ,esto debido a que solo hay 2 muestras de esta clase en el conjunto de prueba. El F1-score weighted de 0.99 confirmaqeu hay un muy buenrendimiento general.

### Matriz de confusión

In [ ]:
cm = confusion_matrix(y_true, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Matriz absoluta
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=class_names, yticklabels=class_names)
axes[0].set_title('Matriz de Confusión')
axes[0].set_ylabel('Real')
axes[0].set_xlabel('Predicho')

# Matriz normalizada
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Greens', ax=axes[1],
            xticklabels=class_names, yticklabels=class_names)
axes[1].set_title('Matriz Normalizada')
axes[1].set_ylabel('Real')
axes[1].set_xlabel('Predicho')

plt.tight_layout()
plt.show()

**Análisis:**

 79/79 Baja correctas, 91/91 Media correctas, y 1/2 Alta correctas.


 El único error es 1 vino Alta confundido con Media, ya que vinos de calidad 8 pueden tener características similares a los de calidad 7. pero  no hay confusiones entre Baja y Alta, lo que indica que el modelo distingue los extremos de calidad.

## 7. Discusión

### Características Importantes y Rendimiento

El análisis de correlación identificó al alcohol como la característica más influyente (+0.455), seguido de acidez volátil (-0.332) , se aplico satisfactoriamente  estas correlaciones en el modelo , logrando 99.42% de accuracy.

El rendimiento fue muy bien  en las clases Baja y Media (100% precision/recall), mientras que la clase Alta mostro algunas  limitaciones (50% recall) debido a que pues hay poquitos datos  (solo 2 muestras en test).

### Impacto del Preprocesamiento y Desafíos

El preprocesamiento fue crucial para el exito pues  la normalización con StandardScaler permitió convergencia rápida como se vio en nel grafico anteriror  (10 épocas) la división mantuvo proporciones de clases, y la conversión a tensores preparo los datos correctamente para PyTorch. Los principales desafíos fueron el desbalance de clases (2% del dataset)  Las decisiones arquitectónicas capas 64 -32, Adam optimizer, early stopping, inicialización Xavier funcionaron  con el preprocesamiento para lograr estos resultados. Para mejorar, depronto mas adelante usaria tecnicas de balanceo mas optimas como (SMOTE, class weights) que realmente no las he usado pero las tengo presentes , y seria chebre recolectar más datos de la clase Alta.